In [7]:
import requests
from bs4 import BeautifulSoup

#load the webpage
r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c=r.content

#create parsed object using BeautifulSoup
soup = BeautifulSoup(c, "html.parser")

#parse using find_all to get divs with class property row .... aka load the listings on the page
all = soup.find_all("div", {"class","propertyRow"})

#look for price of first unit on the page return just the price as a text
all[0].find("h4", {"class","propPrice"}).text.replace("\n", "").replace(" ", "")

'$725,000'

In [10]:
#for loop through each item on the page to return all of the prices
l=[]
base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for item in all:
    #create an empty dictionary to store each objects information
    d={}
    #now add two lines to return the street address on one line and the city/state info on another
    d["Address"] = (item.find_all("span", {"class" , "propAddressCollapse"})[0].text)
    d["Locality"] = (item.find_all("span", {"class" , "propAddressCollapse"})[1].text)
    d["Price"] = (item.find("h4", {"class", "propPrice"}).text.replace("\n", "").replace(" ", ""))
    #now extract number of beds/bath
    #try to pass bed info, but if nothing there then return None
    try:
        d["Beds"] = (item.find("span", {"class" , "infoBed"}).find("b").text)
    except:
        d["Beds"] = (None)
    #try to pass bathroom info, but if nothing there then return None
    try:
        d["Area"] = (item.find("span", {"class" , "infoSqFt"}).find("b").text)
    except:
        d["Area"] = (None)
    #try to pass square footage, but if nothing there then return None
    try:
        d["FullBath"]=(item.find("span", {"class" , "infoValueFullBath"}).find("b").text)
    except:
        d["FullBath"]=(None)
    #try to pass half bath, but if nothing there then return None
    try:
        d["HalfBath"]=(item.find("span", {"class" , "infoValueHalfBath"}).find("b").text)
    except:
        d["HalfBath"]=(None)
    #extract lot size information traversing through for unique identifier (lot size)
    for column_group in item.find_all("div", {"class": "columnGroup"}):
        for feature_group, feature_name in zip(column_group.find_all("span",{"class": "featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
            if "Lot Size" in feature_group.text:
                d["LotSize"]=("Lot size: " + feature_name.text)
    l.append(d)
    print(" ")

In [13]:
#create dataframe
import pandas
df=pandas.DataFrame(l)

In [14]:
df

,Address,Locality,Price,Beds,Area,FullBath,HalfBath,LotSize
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,None,4,None,Lot size: 0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,"3,154",3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,"3,076",3,1,"Lot size: Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,"1,172",3,None,"Lot size: Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4,"1,932",2,None,Lot size: 0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4,"1,676",3,None,"Lot size: Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3,"1,344",2,None,"Lot size: Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3,"1,920",2,None,"Lot size: Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3,"1,664",2,None,Lot size: 2.02 Acres


In [16]:
df.to_csv("Output.csv")